<a href="https://colab.research.google.com/github/TimSim/SimpleChatBot/blob/master/ShakespareChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2. Installing Libraries

Currently, software installations within Google Colaboratory are not persistent, in that you must reinstall libraries every time you (re-)connect to an instance. Since Colab has numerous useful common libraries installed by default, this is less of an issue than it may seem, and installing those libraries which are not pre-installed are easily added in one of a few different ways.


You will want to be aware, however, that installing any software which needs to be built from source may take longer than is feasible when connecting/reconnecting to your instance.

Colab supports both the pip and apt package managers. Regardless of which you are using, remember to prepend any bash commands with a !.

In [3]:
!pip install pandas

In [0]:
# import os
# RaNDOM
import random
#clock training time
import datetime 
# Numpy for vectorization
import numpy as np
# Tensorflow for ML
import tensorflow as tf
# Pandas for file reading/ visualize data
import pandas as pd
# Seaborn as the great data visualizer
import seaborn as sns
#Matplot to visualize data, also Seaborn and pandas do this
import matplotlib.pyplot as plt
# Inline to show images in jupyter notebook
%matplotlib inline

# Set number of columns to show in the notebook
pd.set_option('display.max_columns', 600)
# Set number of rows to show in the notebook
pd.set_option('display.max_rows', 50)
# Make the graphs a bit prettier
#pd.set_option('display.mpl_style', 'default') 

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Shakespeare_data.csv to Shakespeare_data.csv


In [0]:
# Grab the shakespeare data
allData = pd.read_csv('./Shakespeare_data.csv', sep=',')
allData.columns = ["Dataline","Play","PlayerLinenumber","ActSceneLine","Player","PlayerLine"]
allData = list(allData.PlayerLine)

In [4]:
# Converting the dataframe to a single string
textLines = ''.join(allData)
print('text length in number of characters:', len(textLines))

text length in number of characters: 4254892


In [5]:
chars = sorted(list(set(textLines)))
char_size = len(chars)
print('number of characters:', char_size)
print(chars)
# print(textLines)

number of characters: 76
['\t', ' ', '!', '$', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [0]:
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))

In [0]:
#Given a probability of each character, return a likely character, one-hot encoded
def sample(prediction):
    r = random.uniform(0,1)
    s = 0
    char_id = len(prediction) - 1
    for i in range(len(prediction)):
        s += prediction[i]
        if s >= r:
            char_id = i
            break
    char_one_hot = np.zeros(shape=[char_size])
    char_one_hot[char_id] = 1.0
    return char_one_hot

In [8]:
#vectorize our data to feed it into model

len_per_section = 15
skip = 5
sections = []
next_chars = []
#fill sections list with chunks of text, every 10 characters create a new 20 
#character long section
#because we are generating it at a character level
for i in range(0, len(textLines) - len_per_section, skip):
    sections.append(textLines[i: i + len_per_section])
    next_chars.append(textLines[i + len_per_section])
    
print(len(sections), len_per_section, char_size)
    
#Vectorize input and output
#matrix of section length by num of characters
X = np.zeros((len(sections), len_per_section, char_size))
#label column for all the character id's, still zero
y = np.zeros((len(sections), char_size))
#for each char in each section, convert each char to an ID
#for each section convert the labels to ids 

#for i, section in enumerate(sections):
#    for j, char in enumerate(section):
#        X[i, j, char2id[char]] = 1
#    y[i, char2id[next_chars[i]]] = 1
#print(y)

850976 15 76


In [9]:
batch_size = 512
max_steps = 20000
log_every = 2000
test_every = 2000
hidden_nodes = 1024
test_start = 'Will '
checkpoint_directory = 'ckpt'

#Create a checkpoint directory
if tf.gfile.Exists(checkpoint_directory):
    tf.gfile.DeleteRecursively(checkpoint_directory)
tf.gfile.MakeDirs(checkpoint_directory)

print('training data size:', len(X))
print('approximate steps per epoch:', int(len(X)/batch_size))

training data size: 850976
approximate steps per epoch: 1662


In [10]:
#build our model time
#create computation graph
graph = tf.Graph()
#if multiple graphs, but none here jsut one
with graph.as_default():
    ###########
    #Prep
    ###########
    #Variables and placeholders
    #global_step refer to the number of batches seen by the graph. 
    #Everytime a batch is provided, the weights are updated in the 
    #direction that minimizes the loss. global_step just keeps track 
    #of the number of batches seen so far starts off as 0
    global_step = tf.Variable(0)
    
    #data tensor shape feeding in sections
    data = tf.placeholder(tf.float32, [batch_size, len_per_section, char_size])
    #labels
    labels = tf.placeholder(tf.float32, [batch_size, char_size])
    
    #An LSTM RNN (Long Short Term Memory), consists of 3 gates and an internal state, 
    #This enables the LSTM to capture long-term dependencies. 
    #http://suriyadeepan.github.io/2017-02-13-unfolding-rnn-2/
    #lets build weights and biases for each of the 3 gates and then for the cell state
    
    #tf variables
    #Since we need the weights and biases for our model. 
    #We could imagine treating these like additional inputs, 
    #but TensorFlow has an even better way to handle it: Variable
    #A Variable is a modifiable tensor that lives in TensorFlow's graph of 
    #interacting operations. It can be used and even modified by the computation. 
    #For machine learning applications, one generally has the model parameters be Variables.
    
    #Prep LSTM Operation
    #Input gate: weights for input, weights for previous output, and bias
    
    #tf truncated normal
    #Outputs random values from a truncated normal distribution.
    #The generated values follow a normal distribution with specified mean and 
    #standard deviation, except that values whose magnitude is more than 2 standard deviations
    #from the mean are dropped and re-picked.
    #basically randomly initialized values here
    
    #biases act as an anchor

    w_ii = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_io = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_i = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Forget gate: weights for input, weights for previous output, and bias
    w_fi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_fo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_f = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Output gate: weights for input, weights for previous output, and bias
    w_oi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_oo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_o = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Memory cell: weights for input, weights for previous output, and bias
    w_ci = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_co = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_c = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    
    #LSTM Cell
    # given input, output, external state, it will return output and state
    #output starts off empty, LSTM cell calculates it
    
    #Since, we have two kinds of states - the internal state ct 
    #and the (exposed) external state st, and since we need both of 
    #them for the subsequent sequential operations, we combine them 
    #into a tensor at each step, and pass them as input to the next 
    #step. This tensor is unpacked into st_1 and ct_1 at the beginning of each step.
    
    
    def lstm(i, o, state):
        
        #these are all calculated seperately, no overlap until....
        #(input * input weights) + (output * weights for previous output) + bias
        input_gate = tf.sigmoid(tf.matmul(i, w_ii) + tf.matmul(o, w_io) + b_i)
        #(input * forget weights) + (output * weights for previous output) + bias
        forget_gate = tf.sigmoid(tf.matmul(i, w_fi) + tf.matmul(o, w_fo) + b_f)
        #(input * output weights) + (output * weights for previous output) + bias
        output_gate = tf.sigmoid(tf.matmul(i, w_oi) + tf.matmul(o, w_oo) + b_o)
        #(input * internal state weights) + (output * weights for previous output) + bias
        memory_cell = tf.sigmoid(tf.matmul(i, w_ci) + tf.matmul(o, w_co) + b_c)
        
        #...now! multiply forget gate * given state    +  input gate * hidden state
        state = forget_gate * state + input_gate * memory_cell
        #squash that state with tanh nonlin (Computes hyperbolic tangent of x element-wise)
        #multiply by output
        output = output_gate * tf.tanh(state)
        #return 
        return output, state
    
    ###########
    #Operation
    ###########
    #LSTM
    #both start off as empty, LSTM will calculate this
    output = tf.zeros([batch_size, hidden_nodes])
    state = tf.zeros([batch_size, hidden_nodes])

    #unrolled LSTM loop
    #for each input set
    for i in range(len_per_section):
        #calculate state and output from LSTM
        output, state = lstm(data[:, i, :], output, state)
        #to start, 
        if i == 0:
            #store initial output and labels
            outputs_all_i = output
            labels_all_i = data[:, i+1, :]
        #for each new set, concat outputs and labels
        elif i != len_per_section - 1:
            #concatenates (combines) vectors along a dimension axis, not multiply
            outputs_all_i = tf.concat([outputs_all_i, output], 0)
            labels_all_i = tf.concat([labels_all_i, data[:, i+1, :]], 0)
        else:
            #final store
            outputs_all_i = tf.concat([outputs_all_i, output], 0)
            labels_all_i = tf.concat([labels_all_i, labels], 0)
        
    #Classifier
    #The Classifier will only run after saved_output and saved_state were assigned.
    
    #calculate weight and bias values for the network
    #generated randomly given a size and distribution
    w = tf.Variable(tf.truncated_normal([hidden_nodes, char_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([char_size]))
    #Logits simply means that the function operates on the unscaled output 
    #of earlier layers and that the relative scale to understand the units 
    #is linear. It means, in particular, the sum of the inputs may not equal 1, 
    #that the values are not probabilities (you might have an input of 5).
    logits = tf.matmul(outputs_all_i, w) + b
    
    #logits is our prediction outputs, lets compare it with our labels
    #cross entropy since multiclass classification
    #computes the cost for a softmax layer
    #then Computes the mean of elements across dimensions of a tensor.
    #average loss across all values
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels_all_i, logits=logits))

    #Optimizer
    #minimize loss with graident descent, learning rate 10,  keep track of batches
    optimizer = tf.train.GradientDescentOptimizer(10.).minimize(loss, global_step=global_step)
    
    ###########
    #Test
    ###########
    test_data = tf.placeholder(tf.float32, shape=[1, char_size])
    test_output = tf.Variable(tf.zeros([1, hidden_nodes]))
    test_state = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    #Reset at the beginning of each test
    reset_test_state = tf.group(test_output.assign(tf.zeros([1, hidden_nodes])), 
                                test_state.assign(tf.zeros([1, hidden_nodes])))

    #LSTM
    test_output, test_state = lstm(test_data, test_output, test_state)
    test_prediction = tf.nn.softmax(tf.matmul(test_output, w) + b)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [11]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    offset = 0
    saver = tf.train.Saver()
    TF_MIN_GPU_MULTIPROCESSOR_COUNT=4
    for step in range(max_steps):
        offset = offset % len(X)
        if offset <= (len(X) - batch_size):
            batch_data = X[offset: offset + batch_size]
            batch_labels = y[offset: offset + batch_size]
            offset += batch_size
        else:
            to_add = batch_size - (len(X) - offset)
            batch_data = np.concatenate((X[offset: len(X)], X[0: to_add]))
            batch_labels = np.concatenate((y[offset: len(X)], y[0: to_add]))
            offset = to_add
        _, training_loss = sess.run([optimizer, loss], feed_dict={data: batch_data, labels: batch_labels})
        
        if step % log_every == 0:
            print('training loss at step %d: %.2f (%s)' % (step, training_loss, datetime.datetime.now()))

            if step % log_every == 0:
                reset_test_state.run()
                test_generated = test_start
                
                for i in range(len(test_start) - 1):
                    test_X = np.zeros((1, char_size))
                    test_X[0, char2id[test_start[i]]] = 1.
                    _ = sess.run(test_prediction, feed_dict={test_data: test_X})
                
                test_X = np.zeros((1, char_size))
                test_X[0, char2id[test_start[-1]]] = 1.
                
                for i in range(500):
                    prediction = test_prediction.eval({test_data: test_X})[0]
                    next_char_one_hot = sample(prediction)
                    next_char = id2char[np.argmax(next_char_one_hot)]
                    test_generated += next_char
                    test_X = next_char_one_hot.reshape((1, char_size))
                    
                print('=' * 80)
                print(test_generated)
                print('=' * 80)
                
                saver.save(sess, checkpoint_directory + '/model', global_step=step)

training loss at step 0: 0.00 (2018-11-01 20:32:41.785405)
Will 96OBU5H-fz?,LEthyh8SG]h4WHT,NFGkRef'1c4jMFRb]4nwI0]52OgOPCIolM6fOX?E0y	Xo$:Qs	ZGRsaN3AbAjEi$23KsrjVH2.0BEPYywujFVcZUGdhL-ZzW]K3 EdNSB'jl2I(3t]p$l[CI6CaNWX8q3bP!]jf,S8A5IOGgoe'Cq96bdjo$k0tT.G-	Mw!Ug]9HI]U2I3AjxWofB30r:	[f2i!'eylpi5y-J5(C'j !zfhaa'2GzME-,jWu-U[ETEsHoS92f(j3[wSd'Jei5Uhyq[Z)Cg1pVEt2Vna:M)hi Vu7,ljQ!0Y'TywNbIG'KiGJbg-91acTf'5doMzg!l1BRWW7faS1l[SX.4il,JUc)EvAYqTlJrtgO-[2Npj WhkNX9oO-Xq,Bt5T:sQv(E  jU9Z7IiIhACZDT-VxFYvUdOugHtn1vpWSS[OF$gX1[pru$XF]zTKyy$m2] l5FrEZIeUzB([NBif6P'HrBvPl5?A
training loss at step 2000: 0.00 (2018-11-01 20:37:13.893014)
Will QQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ

In [0]:
rm ckpt/model-30*

In [12]:
ls -la ckpt

total 91952
drwxr-xr-x 2 root root     4096 Nov  1 21:13 ./
drwxr-xr-x 1 root root     4096 Nov  1 20:32 ../
-rw-r--r-- 1 root root      247 Nov  1 21:13 checkpoint
-rw-r--r-- 1 root root 18358580 Nov  1 20:55 model-10000.data-00000-of-00001
-rw-r--r-- 1 root root      626 Nov  1 20:55 model-10000.index
-rw-r--r-- 1 root root   459959 Nov  1 20:55 model-10000.meta
-rw-r--r-- 1 root root 18358580 Nov  1 20:59 model-12000.data-00000-of-00001
-rw-r--r-- 1 root root      626 Nov  1 20:59 model-12000.index
-rw-r--r-- 1 root root   459959 Nov  1 20:59 model-12000.meta
-rw-r--r-- 1 root root 18358580 Nov  1 21:04 model-14000.data-00000-of-00001
-rw-r--r-- 1 root root      626 Nov  1 21:04 model-14000.index
-rw-r--r-- 1 root root   459959 Nov  1 21:04 model-14000.meta
-rw-r--r-- 1 root root 18358580 Nov  1 21:08 model-16000.data-00000-of-00001
-rw-r--r-- 1 root root      626 Nov  1 21:08 model-16000.index
-rw-r--r-- 1 root root   459959 Nov  1 21:08 model-16000.meta
-rw-r--r-- 1 root root 1835

In [13]:
# To try your own text uncomment this!

test_start = 'To be or not to be'

with tf.Session(graph=graph) as sess:
    #init graph, load model
    tf.global_variables_initializer().run()
    model = tf.train.latest_checkpoint(checkpoint_directory)
    saver = tf.train.Saver()
    saver.restore(sess, "./ckpt/model-18000")

    #set input variable to generate chars from
    reset_test_state.run() 
    test_generated = test_start

    #for every char in the input sentennce
    for i in range(len(test_start) - 1):
        #initialize an empty char store
        test_X = np.zeros((1, char_size))
        #store it in id from
        test_X[0, char2id[test_start[i]]] = 1.
        #feed it to model, test_prediction is the output value
        _ = sess.run(test_prediction, feed_dict={test_data: test_X})

    
    #where we store encoded char predictions
    test_X = np.zeros((1, char_size))
    test_X[0, char2id[test_start[-1]]] = 1.

    #lets generate 500 characters
    for i in range(500):
        #get each prediction probability
        prediction = test_prediction.eval({test_data: test_X})[0]
        #one hot encode it
        next_char_one_hot = sample(prediction)
        #get the indices of the max values (highest probability)  and convert to char
        next_char = id2char[np.argmax(next_char_one_hot)]
        #add each char to the output text iteratively
        test_generated += next_char
        #update the 
        test_X = next_char_one_hot.reshape((1, char_size))

    print(test_generated)

INFO:tensorflow:Restoring parameters from ./ckpt/model-18000
To be or not to beHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
